# Imports

In [1]:
import sys
import copy
import numpy as np
import pandas as pd
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci, logger
from utils import BasicStatistics, RemovePolicy, AddPolicyInfectedRate, Propaganda, setup_city_layout, confidence_interval
from model.utils import SocialPolicy
from model.debugutils import DebugUtils

# Parameters settings

Default parameters. Unless explicitly overriden, these values are used in all scenarios

In [2]:
basic_parameters = SimulationParameters(
    hospitalization_capacity = 1.0,
    icu_capacity = 1.0
)

# Simulation setup

In [3]:
population_size = 1000
simulation_cycles = 90 # days
multiple_runs = 30

## Setup of monitored variables

In [4]:
basic_r0 = {}
deaths_by_age_group = {}
deaths_total = {}
max_hospitalized = {}
max_icu = {}

In [5]:
trigger_infected_rate = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]

# Run simulation multiple times

In [6]:
for infected_rate in trigger_infected_rate:
    basic_r0[infected_rate] = []
    deaths_by_age_group[infected_rate] = [[], [], [], [], [], [], [], [], [], []]
    deaths_total[infected_rate] = []
    max_hospitalized[infected_rate] = []
    max_icu[infected_rate] = []
    for k in range(multiple_runs):
        params = copy.deepcopy(basic_parameters)
        set_parameters(params)
        model = CovidModel()
        setup_city_layout(model, population_size)
        model.add_listener(AddPolicyInfectedRate(model, SocialPolicy.LOCKDOWN_ALL, infected_rate))
        statistics = BasicStatistics(model)
        model.add_listener(statistics)
        debug = DebugUtils(model)
        logger().model = model
        model.add_listener(debug)
        for i in range(simulation_cycles):
            model.step()
        basic_r0[infected_rate].append(debug.get_R0_stats()['infections'].mean())
        max_hospitalized[infected_rate].append(debug.max_hospitalized / population_size)
        max_icu[infected_rate].append(debug.max_icu / population_size)
        stats = debug.get_age_group_stats()
        deaths_total[infected_rate].append(stats['deaths'].sum() / (stats['deaths'].sum() + stats['recovered'].sum()))
        for i in range(10):
            infected = stats['deaths'][i] + stats['recovered'][i]
            rate = 0 if infected == 0 else stats['deaths'][i] / infected
            deaths_by_age_group[infected_rate][i].append(rate)
        print(infected_rate)
        print(f"Deaths: {confidence_interval(deaths_total[infected_rate])}")
        print(f"Hospitalization: {confidence_interval(max_hospitalized[infected_rate])}")
        print(f"ICU: {confidence_interval(max_icu[infected_rate])}")

0.05
Deaths: (nan, 0.02512562814070352, nan)
Hospitalization: (nan, 0.04, nan)
ICU: (nan, 0.011, nan)


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.05
Deaths: (-0.06430048286041472, 0.032764834272371965, 0.12983015140515863)
Hospitalization: (-0.1861523781511296, 0.0235, 0.23315237815112957)
ICU: (-0.057531023682160475, 0.006, 0.06953102368216048)
0.05
Deaths: (0.002141597324059087, 0.028509889514914646, 0.054878181705770204)
Hospitalization: (-0.017661211507158662, 0.023333333333333334, 0.06432787817382533)
ICU: (-0.006169885625888433, 0.006333333333333333, 0.018836552292555098)
0.05
Deaths: (0.012915908822540645, 0.027264770077362454, 0.041613631332184264)
Hospitalization: (0.0029586730409508523, 0.02525, 0.047541326959049154)
ICU: (8.949581311040733e-05, 0.0072499999999999995, 0.014410504186889592)
0.05
Deaths: (0.016408817413040365, 0.026398972025192712, 0.03638912663734506)
Hospitalization: (0.007854838803708415, 0.023600000000000003, 0.03934516119629159)
ICU: (0.0021115639506332773, 0.006999999999999999, 0.011888436049366721)
0.05
Deaths: (0.01482088967819338, 0.024221365576549483, 0.03362184147490559)
Hospitalization: (0.

0.1
Deaths: (0.013321370127027821, 0.01685727928218618, 0.02039318843734454)
Hospitalization: (0.025357266339357386, 0.032777777777777774, 0.04019828921619816)
ICU: (0.007156798779386248, 0.009000000000000001, 0.010843201220613755)
0.1
Deaths: (0.013457677718285608, 0.018755780744648568, 0.02405388377101153)
Hospitalization: (0.026469714765711236, 0.032999999999999995, 0.03953028523428875)
ICU: (0.007546118884474668, 0.0093, 0.01105388111552533)
0.1
Deaths: (0.014149927980409673, 0.018878028178095318, 0.023606128375780962)
Hospitalization: (0.026233234765524152, 0.03227272727272727, 0.03831221977993039)
ICU: (0.007597197654961241, 0.009181818181818182, 0.010766438708675122)
0.1
Deaths: (0.015072910161374938, 0.0197354147188096, 0.02439791927624426)
Hospitalization: (0.026885611553153707, 0.03233333333333333, 0.03778105511351296)
ICU: (0.00763806037277172, 0.009083333333333334, 0.010528606293894948)
0.1
Deaths: (0.015946219878556534, 0.020537491509134972, 0.02512876313971341)
Hospitaliz

0.15
Deaths: (0.018291420188091, 0.023685829596471586, 0.02908023900485217)
Hospitalization: (0.041872468597473705, 0.0463125, 0.050752531402526294)
ICU: (0.011107380791759406, 0.012937500000000001, 0.014767619208240596)
0.15
Deaths: (0.018248051735726814, 0.023340626204775564, 0.028433200673824315)
Hospitalization: (0.0423089780512994, 0.04647058823529412, 0.050632198419288836)
ICU: (0.011096756506985585, 0.012823529411764708, 0.01455030231654383)
0.15
Deaths: (0.018874703712855545, 0.023719614359679113, 0.02856452500650268)
Hospitalization: (0.04269494317187881, 0.04661111111111112, 0.050527279050343425)
ICU: (0.011087916652682992, 0.012722222222222225, 0.014356527791761458)
0.15
Deaths: (0.019229002039319702, 0.02379527848308561, 0.028361554926851516)
Hospitalization: (0.04322189131074252, 0.04700000000000001, 0.050778108689257494)
ICU: (0.011010412033470323, 0.012578947368421056, 0.014147482703371789)
0.15
Deaths: (0.018929444218206783, 0.023346255299672068, 0.027763066381137354)
H

...

0.25
Deaths: (0.01879452866958649, 0.021075103798099574, 0.02335567892661266)
Hospitalization: (0.05613710419800268, 0.06163333333333334, 0.067129562468664)
ICU: (0.014376797393541491, 0.016000000000000004, 0.017623202606458516)
0.3
Deaths: (nan, 0.015723270440251572, nan)
Hospitalization: (nan, 0.07, nan)
ICU: (nan, 0.014, nan)
0.3
Deaths: (0.014290469423665724, 0.01584566715625353, 0.01740086488884134)
Hospitalization: (-0.06706204736432098, 0.060000000000000005, 0.18706204736432097)
ICU: (0.0071468976317839465, 0.0135, 0.019853102368216052)
0.3
Deaths: (0.00012849087536210418, 0.0206039387467393, 0.0410793866181165)
Hospitalization: (0.003826824215898847, 0.07566666666666667, 0.14750650911743451)
ICU: (-0.004697163450459483, 0.019, 0.042697163450459485)
0.3
Deaths: (0.010278138005249494, 0.021105127973097954, 0.031932117940946414)
Hospitalization: (0.039700302137072575, 0.078, 0.11629969786292743)
ICU: (0.006606152277389208, 0.019, 0.03139384772261079)
0.3
Deaths: (0.011080759972606

0.35
Deaths: (0.01754286705858165, 0.022632731040820656, 0.02772259502305966)
Hospitalization: (0.061197532055122274, 0.07166666666666667, 0.08213580127821106)
ICU: (0.015343197248356161, 0.018444444444444447, 0.021545691640532733)
0.35
Deaths: (0.018444358702677854, 0.0229781535889125, 0.027511948475147145)
Hospitalization: (0.06300311200246003, 0.0723, 0.08159688799753997)
ICU: (0.015677045934663722, 0.018400000000000003, 0.021122954065336284)
0.35
Deaths: (0.018075553969422443, 0.02235026949641396, 0.02662498502340548)
Hospitalization: (0.06416449885088732, 0.07245454545454545, 0.08074459205820358)
ICU: (0.01524793718931815, 0.017909090909090913, 0.020570244628863678)
0.35
Deaths: (0.01821790990786801, 0.02210901811490346, 0.026000126321938908)
Hospitalization: (0.06399265306917867, 0.07166666666666666, 0.07934068026415464)
ICU: (0.015591987748201652, 0.018000000000000002, 0.020408012251798353)
0.35
Deaths: (0.01667188900803125, 0.020982377226615827, 0.025292865445200402)
Hospitaliz

0.4
Deaths: (0.01623888041056682, 0.02031540256410272, 0.024391924717638618)
Hospitalization: (0.0689423719425916, 0.07881250000000001, 0.08868262805740841)
ICU: (0.017031000406669237, 0.019437500000000003, 0.02184399959333077)
0.4
Deaths: (0.015765794331117387, 0.019755165172477548, 0.02374453601383771)
Hospitalization: (0.06954299346852416, 0.07876470588235296, 0.08798641829618176)
ICU: (0.01709753686175027, 0.019352941176470594, 0.02160834549119092)
0.4
Deaths: (0.01588396398967845, 0.019635746763051954, 0.023387529536425457)
Hospitalization: (0.07017918888330679, 0.07883333333333335, 0.08748747778335991)
ICU: (0.01727123543341415, 0.019388888888888893, 0.021506542344363635)
0.4
Deaths: (0.01632473539976829, 0.019902902031830892, 0.023481068663893493)
Hospitalization: (0.07102060291669293, 0.0792105263157895, 0.08740044971488606)
ICU: (0.017595285515320434, 0.019684210526315794, 0.021773135537311153)
0.4
Deaths: (0.016405138586263277, 0.01979452364238123, 0.02318390869849918)
Hospit

0.45
Deaths: (0.019887357617964774, 0.022287965184247405, 0.024688572750530036)
Hospitalization: (0.07901684175621104, 0.08573913043478261, 0.09246141911335418)
ICU: (0.01872998538156916, 0.02121739130434783, 0.0237047972271265)
0.45
Deaths: (0.020279095448635827, 0.02284527238858897, 0.025411449328542113)
Hospitalization: (0.07948625470123585, 0.08591666666666666, 0.09234707863209746)
ICU: (0.019031086365619246, 0.021458333333333333, 0.02388558030104742)
0.45
Deaths: (0.02055653438802234, 0.023047467366760537, 0.025538400345498733)
Hospitalization: (0.08005027679903896, 0.08623999999999998, 0.092429723200961)
ICU: (0.019227764184577268, 0.021560000000000003, 0.023892235815422738)
0.45
Deaths: (0.02080188343719848, 0.023214767093961, 0.02562765075072352)
Hospitalization: (0.08078193272538536, 0.08684615384615382, 0.09291037496692228)
ICU: (0.01934064534786844, 0.02157692307692308, 0.02381320080597772)
0.45
Deaths: (0.021076762025151324, 0.023439727680439122, 0.02580269333572692)
Hospit

0.5
Deaths: (0.0206774100457348, 0.023126281033890406, 0.02557515202204601)
Hospitalization: (0.08834785672112765, 0.09463333333333335, 0.10091880994553905)
ICU: (0.021736080112873066, 0.024000000000000007, 0.02626391988712695)


# Print results

In [7]:
for infected_rate in trigger_infected_rate:
    print(confidence_interval(basic_r0[infected_rate]))

(1.5252689533065753, 1.715669007718527, 1.9060690621304788)
(1.619320721829899, 1.826969946679822, 2.034619171529745)
(1.7301862345772938, 1.9181952958678783, 2.1062043571584628)
(1.7231744907763857, 1.9178158659886515, 2.112457241200917)
(1.8464308889930838, 2.0541556161414887, 2.2618803432898935)
(1.8926093615449375, 2.060801376376623, 2.228993391208308)
(1.8727148456645297, 2.052574176105002, 2.2324335065454743)
(1.7412906874623835, 1.918298419586829, 2.095306151711274)
(1.8185726042759829, 1.9801312514381595, 2.141689898600336)
(1.8845523160337805, 2.050692557619131, 2.216832799204482)


In [8]:
for infected_rate in trigger_infected_rate:
    print(f"Total: {confidence_interval(deaths_total[infected_rate])}")
    for i in range(10):
        print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(deaths_by_age_group[infected_rate][i])}")

Total: (0.01640413193864749, 0.019553234623428364, 0.02270233730820924)
0 to 9: (0.0, 0.0, 0.0)
10 to 19: (-0.0003196420923953221, 0.00030581039755351685, 0.0009312628875023559)
20 to 29: (1.67418616530534e-05, 0.004516327003868532, 0.00901591214608401)
30 to 39: (0.0024112374973623615, 0.008835196893384446, 0.015259156289406531)
40 to 49: (0.013017229838778671, 0.0251782247615581, 0.03733921968433753)
50 to 59: (0.036701780672481685, 0.08376305658914356, 0.13082433250580544)
60 to 69: (0.18165427123907713, 0.29389601139601135, 0.40613775155294557)
70 to 79: (0.17182995355226915, 0.33888888888888885, 0.5059478242255085)
80 to 89: (0.04808415598177479, 0.2, 0.35191584401822523)
90 to 99: (-0.03484098807109009, 0.03333333333333333, 0.10150765473775675)
Total: (0.01845121862643215, 0.02116654496845986, 0.02388187131048757)
0 to 9: (0.0, 0.0, 0.0)
10 to 19: (0.0, 0.0, 0.0)
20 to 29: (0.00044446401446869436, 0.0035123912415294086, 0.006580318468590123)
30 to 39: (0.00505321788136671, 0.0101

In [9]:
for infected_rate in trigger_infected_rate:
    print(infected_rate)
    print(f"Deaths: {confidence_interval(deaths_total[infected_rate])}")
    print(f"Hospitalization: {confidence_interval(max_hospitalized[infected_rate])}")
    print(f"ICU: {confidence_interval(max_icu[infected_rate])}")

0.05
Deaths: (0.01640413193864749, 0.019553234623428364, 0.02270233730820924)
Hospitalization: (0.018272657760261723, 0.022100000000000005, 0.025927342239738287)
ICU: (0.005155237090895498, 0.006333333333333335, 0.007511429575771172)
0.1
Deaths: (0.01845121862643215, 0.02116654496845986, 0.02388187131048757)
Hospitalization: (0.029383411389492027, 0.033433333333333336, 0.037483255277174644)
ICU: (0.007853577931306453, 0.008866666666666667, 0.009879755402026881)
0.15
Deaths: (0.017643495369134565, 0.02118948209492448, 0.024735468820714396)
Hospitalization: (0.03779764069469845, 0.04196666666666667, 0.046135692638634894)
ICU: (0.009999131054362426, 0.011366666666666669, 0.012734202278970912)
0.2
Deaths: (0.01753173829430339, 0.0205678734721418, 0.023604008649980208)
Hospitalization: (0.04466393574798523, 0.0494, 0.05413606425201477)
ICU: (0.011403175659020012, 0.01286666666666667, 0.014330157674313329)
0.25
Deaths: (0.01879452866958649, 0.021075103798099574, 0.02335567892661266)
Hospital